In [32]:
%matplotlib inline
from sklearn import datasets
import matplotlib.pyplot as plt

iris = datasets.load_iris()

In [55]:
import tensorflow as tf
tf.reset_default_graph()

n_input = len(iris.data[0])
n_output = 3 # [0,1,2]... set(iris.target)

input_shape = [None,n_input]
inputplaceholder = tf.placeholder(dtype=tf.float32, shape=input_shape, name="input_placeholder")

weights = tf.Variable(tf.random_normal([n_input, n_output]), name="weights")
biases = tf.Variable(tf.zeros([n_output]), name="biases")

print(weights)
print(biases)

layer_1 = tf.matmul(inputplaceholder, weights)
layer_2 = tf.add(layer_1, biases)
outputlayer = tf.nn.sigmoid(layer_2)

print(outputlayer)

learning_rate = 0.1

labelsplaceholder = tf.placeholder(dtype=tf.float32, shape=[None,n_output], name="labels_placeholder")
#cost = tf.losses.mean_squared_error(labelsplaceholder, outputlayer)
cross_entropy_cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=labelsplaceholder,logits=outputlayer)
cost = tf.reduce_mean(cross_entropy_cost)

print(cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

<tf.Variable 'weights:0' shape=(4, 3) dtype=float32_ref>
<tf.Variable 'biases:0' shape=(3,) dtype=float32_ref>
Tensor("Sigmoid:0", shape=(?, 3), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [61]:
import random
from sklearn import preprocessing

batch_size = 10
iterations = 400

scaler = preprocessing.StandardScaler().fit(iris.data)
scaled_data = scaler.transform(iris.data)
mydata = list(zip(scaled_data, iris.target))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

history_loss = list()
for _ in range(iterations):
    inputdata = list()
    output_data = list()
    for _ in range(batch_size):
        input_output_pairs = random.choice(mydata)
        inputdata.append(input_output_pairs[0])
        output_one_hot = [0.0,0.0,0.0]
        output_one_hot[input_output_pairs[1]] = 1.0
        output_data.append(output_one_hot)

    feed_dict={inputplaceholder: inputdata, labelsplaceholder: output_data}
    res_optimizer, res_cost = sess.run([optimizer, cost], feed_dict=feed_dict)
    #print(res_cost)
    history_loss.append(res_cost)
    
    #stampa il dettaglio per debug 
    #print(labelsplaceholder.eval(feed_dict, sess))
    #print(outputlayer.eval(feed_dict, sess))
    #print(cross_entropy_cost.eval(feed_dict, sess))
    #print(cost.eval(feed_dict, sess))
    #print("---------------------------------------------")
    

In [45]:
#esempio minimale di esempio
import tensorflow as tf

sess = tf.InteractiveSession()

p = tf.placeholder(tf.float32, shape=[None, 2])
logit_q = tf.placeholder(tf.float32, shape=[None, 2])
q = tf.nn.softmax(logit_q)

feed_dict = {
  p: [[0, 1],
      [1, 0],
      [1, 0]],
  logit_q: [[0.2, 0.8],
            [0.7, 0.3],
            [0.4, 0.6]]
}

prob_sigmoid_crossentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=p, logits=logit_q)
prob_sigmoid_crossentropy_homemade = p * -tf.log(tf.sigmoid(logit_q)) + (1-p) * -tf.log(1-tf.sigmoid(logit_q))
prob_sigmoid_crossentropy_mean = tf.reduce_mean(prob_sigmoid_crossentropy)
prob_sigmoid_crossentropy_sum = tf.reduce_sum(prob_sigmoid_crossentropy)
prob_softmax_crossentropy = tf.nn.softmax_cross_entropy_with_logits(labels=p, logits=logit_q)
prob_softmax_cross_entropy_with_logits_handmade = -tf.reduce_sum(p * tf.log(q), axis=1)
prob_softmax_crossentropy_mean = tf.reduce_mean(prob_softmax_crossentropy)
prob_softmax_crossentropy_sum = tf.reduce_sum(prob_softmax_crossentropy)
prob_mean_squared_error = tf.losses.mean_squared_error(p, q)
prob_mean_squared_error_squared_difference_homemade = tf.squared_difference(q, p)
prob_mean_squared_error_squared_difference_mean_homemade = tf.reduce_mean(tf.squared_difference(q, p)) 
print("target: ")
print(p.eval(feed_dict))
print()
print("logit: ")
print(q.eval(feed_dict))
print()
print("sigmoid_cross_entropy_with_logits: ")
print(prob_sigmoid_crossentropy.eval(feed_dict))
print()
print("sigmoid_cross_entropy_with_logits a mano: ")
print(prob_sigmoid_crossentropy_homemade.eval(feed_dict))
print()
print("sigmoid_cross_entropy_with_logits + reduce_mean: ")
print(prob_sigmoid_crossentropy_mean.eval(feed_dict))
print()
print("sigmoid_cross_entropy_with_logits + reduce_sum: ")
print(prob_sigmoid_crossentropy_sum.eval(feed_dict))
print()
print("softmax_cross_entropy_with_logits: ")
print(prob_softmax_crossentropy.eval(feed_dict))
print()
print("softmax_cross_entropy_with_logits a mano: ")
print(prob_softmax_cross_entropy_with_logits_handmade.eval(feed_dict))
print()
print("softmax_cross_entropy_with_logits + reduce_mean: ")
print(prob_softmax_crossentropy_mean.eval(feed_dict))
print()
print("softmax_cross_entropy_with_logits + reduce_sum: ")
print(prob_softmax_crossentropy_sum.eval(feed_dict))
print()
print("mean_squared_error: ")
print(prob_mean_squared_error.eval(feed_dict))
print()
print("mean_squared_error a mano: ")
print(prob_mean_squared_error_squared_difference_homemade.eval(feed_dict))
print(prob_mean_squared_error_squared_difference_mean_homemade.eval(feed_dict))
print()

C:\Users\Utente\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


target: 
[[0. 1.]
 [1. 0.]
 [1. 0.]]

logit: 
[[0.35434368 0.6456563 ]
 [0.59868765 0.40131232]
 [0.450166   0.549834  ]]

sigmoid_cross_entropy_with_logits: 
[[0.79813886 0.3711007 ]
 [0.40318602 0.8543552 ]
 [0.5130153  1.037488  ]]

sigmoid_cross_entropy_with_logits a mano: 
[[0.7981388  0.37110066]
 [0.40318608 0.8543552 ]
 [0.5130153  1.0374879 ]]

sigmoid_cross_entropy_with_logits + reduce_mean: 
0.66288066

sigmoid_cross_entropy_with_logits + reduce_sum: 
3.977284

softmax_cross_entropy_with_logits: 
[0.437488   0.5130153  0.79813886]

softmax_cross_entropy_with_logits a mano: 
[0.43748796 0.5130153  0.7981389 ]

softmax_cross_entropy_with_logits + reduce_mean: 
0.5828807

softmax_cross_entropy_with_logits + reduce_sum: 
1.7486421

mean_squared_error: 
0.19630949

mean_squared_error a mano: 
[[0.12555945 0.12555946]
 [0.1610516  0.16105159]
 [0.30231744 0.30231744]]
0.19630949

